`pip install tensorflow-addons`

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import datetime
import tensorflow_addons as tfa
PAD_ID = 0

class DateData:
    def __init__(self, n):
        np.random.seed(1)
        self.date_cn = []
        self.date_en = []
        for timestamp in np.random.randint(143835585, 2043835585, n):
            date = datetime.datetime.fromtimestamp(timestamp)
            self.date_cn.append(date.strftime("%y-%m-%d"))
            self.date_en.append(date.strftime("%d/%b/%Y"))
        self.vocab = set(
            [str(i) for i in range(0, 10)] + ["-", "/", "<GO>", "<EOS>"] + [
                i.split("/")[1] for i in self.date_en])
        self.v2i = {v: i for i, v in enumerate(sorted(list(self.vocab)), start=1)}
        self.v2i["<PAD>"] = PAD_ID
        self.vocab.add("<PAD>")
        self.i2v = {i: v for v, i in self.v2i.items()}
        self.x, self.y = [], []
        for cn, en in zip(self.date_cn, self.date_en):
            self.x.append([self.v2i[v] for v in cn])
            self.y.append(
                [self.v2i["<GO>"], ] + [self.v2i[v] for v in en[:3]] + [
                    self.v2i[en[3:6]], ] + [self.v2i[v] for v in en[6:]] + [
                    self.v2i["<EOS>"], ])
        self.x, self.y = np.array(self.x), np.array(self.y)
        self.start_token = self.v2i["<GO>"]
        self.end_token = self.v2i["<EOS>"]

    def sample(self, n=64):
        bi = np.random.randint(0, len(self.x), size=n)
        bx, by = self.x[bi], self.y[bi]
        decoder_len = np.full((len(bx),), by.shape[1] - 1, dtype=np.int32)
        return bx, by, decoder_len

    def idx2str(self, idx):
        x = []
        for i in idx:
            x.append(self.i2v[i])
            if i == self.end_token:
                break
        return "".join(x)

    @property
    def num_word(self):
        return len(self.vocab)

In [2]:
class Seq2Seq(keras.Model):
    def __init__(self, enc_v_dim, dec_v_dim, emb_dim, units, max_pred_len, start_token, end_token):
        super().__init__()
        self.units = units

        # encoder
        self.enc_embeddings = keras.layers.Embedding(
            input_dim=enc_v_dim, output_dim=emb_dim,  # [enc_n_vocab, emb_dim]
            embeddings_initializer=tf.initializers.RandomNormal(0., 0.1),
        )
        self.encoder = keras.layers.LSTM(units=units, return_sequences=True, return_state=True)

        # decoder
        self.dec_embeddings = keras.layers.Embedding(
            input_dim=dec_v_dim, output_dim=emb_dim,  # [dec_n_vocab, emb_dim]
            embeddings_initializer=tf.initializers.RandomNormal(0., 0.1),
        )
        self.decoder_cell = keras.layers.LSTMCell(units=units)
        decoder_dense = keras.layers.Dense(dec_v_dim)
        # train decoder
        self.decoder_train = tfa.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            sampler=tfa.seq2seq.sampler.TrainingSampler(),   # sampler for train
            output_layer=decoder_dense
        )
        # predict decoder
        self.decoder_eval = tfa.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            sampler=tfa.seq2seq.sampler.GreedyEmbeddingSampler(),       # sampler for predict
            output_layer=decoder_dense
        )

        self.cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        self.opt = keras.optimizers.Adam(0.01)
        self.max_pred_len = max_pred_len
        self.start_token = start_token
        self.end_token = end_token

    def encode(self, x):
        embedded = self.enc_embeddings(x)
        init_s = [tf.zeros((x.shape[0], self.units)), tf.zeros((x.shape[0], self.units))]
        o, h, c = self.encoder(embedded, initial_state=init_s)
        return [h, c]

    def inference(self, x):
        s = self.encode(x)
        done, i, s = self.decoder_eval.initialize(
            self.dec_embeddings.variables[0],
            start_tokens=tf.fill([x.shape[0], ], self.start_token),
            end_token=self.end_token,
            initial_state=s,
        )
        pred_id = np.zeros((x.shape[0], self.max_pred_len), dtype=np.int32)
        for l in range(self.max_pred_len):
            o, s, i, done = self.decoder_eval.step(
                time=l, inputs=i, state=s, training=False)
            pred_id[:, l] = o.sample_id
        return pred_id

    def train_logits(self, x, y, seq_len):
        s = self.encode(x)
        dec_in = y[:, :-1]   # ignore <EOS>
        dec_emb_in = self.dec_embeddings(dec_in)
        o, _, _ = self.decoder_train(dec_emb_in, s, sequence_length=seq_len)
        logits = o.rnn_output
        return logits

    def step(self, x, y, seq_len):
        with tf.GradientTape() as tape:
            logits = self.train_logits(x, y, seq_len)
            dec_out = y[:, 1:]  # ignore <GO>
            loss = self.cross_entropy(dec_out, logits)
            grads = tape.gradient(loss, self.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.trainable_variables))
        return loss.numpy()


In [3]:
data = DateData(4000)
print("Chinese time order: yy/mm/dd ", data.date_cn[:3], "\nEnglish time order: dd/M/yyyy ", data.date_en[:3])
print("vocabularies: ", data.vocab)
print("x index sample: \n{}\n{}".format(data.idx2str(data.x[0]), data.x[0]),
      "\ny index sample: \n{}\n{}".format(data.idx2str(data.y[0]), data.y[0]))

Chinese time order: yy/mm/dd  ['31-04-26', '04-07-18', '33-06-06'] 
English time order: dd/M/yyyy  ['26/Apr/2031', '18/Jul/2004', '06/Jun/2033']
vocabularies:  {'<EOS>', 'Aug', 'Dec', '5', '2', 'Apr', '-', '1', 'Jul', '<GO>', 'Jun', '/', '9', '7', 'Nov', '4', '6', 'Sep', '8', '<PAD>', 'May', '3', '0', 'Mar', 'Jan', 'Feb', 'Oct'}
x index sample: 
31-04-26
[6 4 1 3 7 1 5 9] 
y index sample: 
<GO>26/Apr/2031<EOS>
[14  5  9  2 15  2  5  3  6  4 13]


In [4]:
model = Seq2Seq(
    data.num_word, data.num_word, emb_dim=16, units=32,
    max_pred_len=11, start_token=data.start_token, end_token=data.end_token)

# training
for t in range(1500):
    bx, by, decoder_len = data.sample(32)
    loss = model.step(bx, by, decoder_len)
    if t % 70 == 0:
        target = data.idx2str(by[0, 1:-1])
        pred = model.inference(bx[0:1])
        res = data.idx2str(pred[0])
        src = data.idx2str(bx[0])
        print(
            "t: ", t,
            "| loss: %.3f" % loss,
            "| input: ", src,
            "| target: ", target,
            "| inference: ", res,
        )


t:  0 | loss: 3.302 | input:  96-06-17 | target:  17/Jun/1996 | inference:  //Jul/0/00/0/
t:  70 | loss: 1.096 | input:  91-08-19 | target:  19/Aug/1991 | inference:  16/Jul/2001<EOS>
t:  140 | loss: 0.768 | input:  11-04-30 | target:  30/Apr/2011 | inference:  03/May/2023<EOS>
t:  210 | loss: 0.511 | input:  76-03-14 | target:  14/Mar/1976 | inference:  14/May/1985<EOS>
t:  280 | loss: 0.370 | input:  83-11-19 | target:  19/Nov/1983 | inference:  19/Jan/1984<EOS>
t:  350 | loss: 0.223 | input:  08-07-22 | target:  22/Jul/2008 | inference:  22/Mar/2008<EOS>
t:  420 | loss: 0.151 | input:  05-02-10 | target:  10/Feb/2005 | inference:  10/Aug/2005<EOS>
t:  490 | loss: 0.103 | input:  32-10-02 | target:  02/Oct/2032 | inference:  02/Jan/2032<EOS>
t:  560 | loss: 0.067 | input:  81-06-29 | target:  29/Jun/1981 | inference:  29/Jun/1981<EOS>
t:  630 | loss: 0.032 | input:  76-08-18 | target:  18/Aug/1976 | inference:  18/Aug/1976<EOS>
t:  700 | loss: 0.016 | input:  87-02-16 | target:  16/F